# 다중 회귀 분석 실습
Dataset :  diabetes dataset
Diabetes dataset은 당뇨병 환자들의 1년간 당뇨병 진행 변화에 대한 데이터이다.  
독립 변수는 나이와 성별 등 환자의 정보이며, 예측해야 하는 종속 변수는 당뇨병의 진행 정도이다. 자세한 사항은 이곳에서 확인할 수 있다.  
결측치가 없고 데이터 정규화(normalization)도 완료되어 있어, 제공된 그대로 분석에 사용하면 된다.

In [1]:
from sklearn import datasets
## load dataset
data = datasets.load_diabetes()

In [2]:
# 데이터 확인
for k, v in data.items():
  print(k, ': ', v)

data :  [[ 0.03807591  0.05068012  0.06169621 ... -0.00259226  0.01990842
  -0.01764613]
 [-0.00188202 -0.04464164 -0.05147406 ... -0.03949338 -0.06832974
  -0.09220405]
 [ 0.08529891  0.05068012  0.04445121 ... -0.00259226  0.00286377
  -0.02593034]
 ...
 [ 0.04170844  0.05068012 -0.01590626 ... -0.01107952 -0.04687948
   0.01549073]
 [-0.04547248 -0.04464164  0.03906215 ...  0.02655962  0.04452837
  -0.02593034]
 [-0.04547248 -0.04464164 -0.0730303  ... -0.03949338 -0.00421986
   0.00306441]]
target :  [151.  75. 141. 206. 135.  97. 138.  63. 110. 310. 101.  69. 179. 185.
 118. 171. 166. 144.  97. 168.  68.  49.  68. 245. 184. 202. 137.  85.
 131. 283. 129.  59. 341.  87.  65. 102. 265. 276. 252.  90. 100.  55.
  61.  92. 259.  53. 190. 142.  75. 142. 155. 225.  59. 104. 182. 128.
  52.  37. 170. 170.  61. 144.  52. 128.  71. 163. 150.  97. 160. 178.
  48. 270. 202. 111.  85.  42. 170. 200. 252. 113. 143.  51.  52. 210.
  65. 141.  55. 134.  42. 111.  98. 164.  48.  96.  90. 162. 150

In [3]:
# 데이터 저장 및 확인
x = data['data']
y = data['target']

print(x.shape, y.shape)

(442, 10) (442,)


In [4]:
## Test, Train 분할하기
from sklearn.model_selection import train_test_split

# 데이터 나누기 - 6:2:2 비율
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=1)

print(x_train.shape, x_val.shape, x_test.shape)

(264, 10) (89, 10) (89, 10)


In [5]:
## 모델 학습하기
import numpy as np
from sklearn.linear_model import LinearRegression

# 회귀분석 객체 생성
lr = LinearRegression()

# 회귀분석 모델 학습
lr.fit(x_train, y_train)

LinearRegression()

In [6]:
# 회귀계수 확인 (w,alpah)
print(lr.intercept_)
print(lr.coef_)

147.87388534119617
[ -39.57865915 -236.60328634  489.97259328  352.8678163  -873.03276134
  430.03354622  208.4341436   306.03221956  746.84762599  116.37588728]


In [8]:
## 결정계수 R-Square 확인
# training data 예측값
pred_train = lr.predict(x_train)

In [9]:
# training data에 대한 R-square 계산
from sklearn.metrics import r2_score
print(r2_score(y_train, pred_train))

0.5353262888595558


In [11]:
# validation data에 대한 R-square 계산
pred_val = lr.predict(x_val)
print(r2_score(y_val, pred_val))

0.493408590131314


In [12]:
# test data에 대한 R-square 계산
pred_test = lr.predict(x_test)
print(r2_score(y_test, pred_test))

0.4432722010710277


In [19]:
# 선택된 변수들, R-square 값 및 모델 저장
best_variables = []
best_r2 = 0.
best_model = None

# 남아 있는 변수들
remain_variables = list(range(10))

for round in range(10):
    print(f"============ round {round+1} ============")
    r2_of_this_round = []
    models_of_this_round = []
    
    for var in remain_variables:
    # 사용될 변수들과 모델
        use_vars = best_variables + [var]
        model = LinearRegression()

    # 지정된 변수만 사용하게끔 데이터 추출
        x_train_small = x_train[:, use_vars]
        x_val_small = x_val[:, use_vars]

    # 지정된 변수로 모델 학습
        model.fit(x_train_small, y_train)
        models_of_this_round.append(model)

    # validation R-square 계산
        r2 = r2_score(y_val, model.predict(x_val_small))
        r2_of_this_round.append(r2)
  
  # R-square 가 높은 모델 선택
    best_r2_of_this_round = np.max(r2_of_this_round)

  # 이전 round와 비교
    if best_r2_of_this_round > best_r2:
        best_var_of_this_round = np.argmax(r2_of_this_round)

    # 변수 추가, R-square 값 및 모델 업데이트
        best_variables.append(remain_variables[best_var_of_this_round])
        best_r2 = best_r2_of_this_round
        best_model = models_of_this_round[best_var_of_this_round]

    # 남은 변수들 중 선택된 변수 제거
        remain_variables.pop(best_var_of_this_round)

        print('best variables updated: ', best_variables)
        print('current best r2: ', best_r2)

  # 더 이상 개선되지 않으면 멈춤  
    else:
        print("no improvement")
        break



print('\n---------------------------------------------------\n')
print('final variables: ', sorted(best_variables))
print('final r2: ', best_r2)
  


============ round 1 ============
best variables updated:  [2]
current best r2:  0.4015039066196985
============ round 2 ============
best variables updated:  [2, 8]
current best r2:  0.48019951742375144
============ round 3 ============
best variables updated:  [2, 8, 6]
current best r2:  0.49118013719572606
============ round 4 ============
best variables updated:  [2, 8, 6, 1]
current best r2:  0.5142459076902111
============ round 5 ============
best variables updated:  [2, 8, 6, 1, 3]
current best r2:  0.5333863843789602
============ round 6 ============
no improvement

---------------------------------------------------

final variables:  [1, 2, 3, 6, 8]
final r2:  0.5333863843789602


In [20]:
# 최적 모델 성능
print(r2_score(y_test, best_model.predict(x_test[:, best_variables])))

0.42090782097560653


In [ ]:
# 선택된 변수들, R-square 값 및 모델 저장
best_variables = list(range(10))
best_adj_r2 = 0.
best_model = None


# TODO: 선택된 변수들 중 Adjusted R-square의 손실이 가장 적은 변수를 골라 없애기
for round in range(10):
    print(f"============ round {round+1} ============")
    adj_r2_of_this_round = []
    models_of_this_round = []
    
    for var in best_variables:
        # 사용될 변수들과 모델
        use_vars = best_variables + [var]
        model = LinearRegression()

    # 지정된 변수만 사용하게끔 데이터 추출
        x_train_small = x_train[:, use_vars]
        x_val_small = x_val[:, use_vars]

    # 지정된 변수로 모델 학습
        model.fit(x_train_small, y_train)
        models_of_this_round.append(model)

    # validation R-square 계산
        r2 = r2_score(y_val, model.predict(x_val_small))
        r2_of_this_round.append(r2)
  
    # R-square 가 낮은 모델 선택
    worst_r2_of_this_round = np.min(r2_of_this_round)
    
    # 이전 round와 비교
    if worst_r2_of_this_round < worst_r2:
        worst_r2_of_this_round = np.argmin(r2_of_this_round)

    # 변수 추가, R-square 값 및 모델 업데이트
        best_variables.append(remain_variables[best_var_of_this_round])
        worst_r2 = worst_r2_of_this_round
        best_model = models_of_this_round[best_var_of_this_round]

    # 남은 변수들 중 선택된 변수 제거
        remain_variables.pop(best_var_of_this_round)

        print('best variables updated: ', best_variables)
        print('current best r2: ', best_r2)
    

print('---------------------------------------------------')
print('final variables: ', sorted(best_variables))
print('final adj_r2: ', best_adj_r2)

print('---------------------------------------------------')
test_r2=r2_score(y_test, best_model.predict(x_test[:, best_variables]))
test_adj_r2 =1-(1-test_r2)*(y_test.shape[0]-1)/(y_test.shape[0]-len(best_variables)-1) 
print('test adjust R-square: ', test_adj_r2)